<a href="https://colab.research.google.com/github/chupvl/ez_enaminetor/blob/master/2023_06_17_rdkit_pyarrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enaminator: PyArrow and Dask for dealing with Enamine files

## Checking instance

In [ ]:
!nproc

8


In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           50Gi       760Mi        34Gi       7.0Mi        15Gi        49Gi
Swap:            0B          0B          0B


## Data

In [ ]:
#!wget https://ftp.enamine.net/download/REAL/Enamine_REAL_350-3_lead-like_cxsmiles.cxsmiles.bz2
# !wget https://ftp.enamine.net/download/REAL/Enamine_REAL_lead-like_cxsmiles.cxsmiles.bz2
!wget https://ftp.enamine.net/download/REAL/Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles.bz2

--2023-06-22 19:29:47--  https://ftp.enamine.net/download/REAL/Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles.bz2
Resolving ftp.enamine.net (ftp.enamine.net)... 18.119.77.46
Connecting to ftp.enamine.net (ftp.enamine.net)|18.119.77.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 613714240 (585M) [application/octet-stream]
Saving to: ‘Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles.bz2’

Enamine_Diverse_REA 100%[===================>] 585.28M  15.6MB/s    in 38s     

2023-06-22 19:30:27 (15.2 MB/s) - ‘Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles.bz2’ saved [613714240/613714240]



In [ ]:
%%time
!bzip2 -d Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles.bz2

bzip2: Output file Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles already exists.
CPU times: user 3.26 ms, sys: 2.14 ms, total: 5.4 ms
Wall time: 105 ms


In [ ]:
!head Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles

smiles	idnumber	Type
FCC(CF)NC(CF)CF	PV-009095522958	M
FC(F)(F)C12CC(NC3CSCSC3)(CO1)C2	Z5067467525	M
CCN(CC1C2CC3C(C2)C13)C(C)(C)C	PV-009234181902	M
Br/C=C\CN1C2CC3CC1CC(C2)O3	Z7289498217	M
CC(N[C@H]1[C@@H]2C[C@H]1N[C@@H]2C)C1C(C)(C)C1(C)C |&1:3,4,6,8|	PV-008660388768	M
FC(F)C1(NCC=CCl)COC1	Z4428721271	M
CC(C)C(NC(C)C1C2CC3C(C2)C31)C(C)C	PV-002844111784	M
FC(F)(F)C=CC=C1COC1	PV-003477850680	M
FCC12CC(NCC3CC(F)C3)(CO1)C2	Z4496448804	M


In [ ]:
!wc -l Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles

48232772 Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles


## Libs

In [ ]:
!pip install rdkit -q
!pip install pyarrow -q
!pip install polars -q
!pip install dask -q

In [ ]:
!git clone https://github.com/chupvl/ez_enaminetor
!pip install ./ez_enaminetor/.

fatal: destination path 'ez_enaminetor' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ez_enaminetor
  Preparing metadata (setup.py) ... done
  Created wheel for ez-enaminetor: filename=ez_enaminetor-2023.6-py3-none-any.whl size=3722 sha256=e003462c79b2d3116d3781bf836bf72ad645fb78f7d512893454de2ecbd5dc3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dyuugrp/wheels/a6/fb/e5/26413647301cb325e11583664b3a0a702df412ac53a916df1a
Successfully built ez-enaminetor
  Attempting uninstall: ez-enaminetor
    Found existing installation: ez-enaminetor 2023.6
    Uninstalling ez-enaminetor-2023.6:
      Successfully uninstalled ez-enaminetor-2023.6


## Preparation: 1M chunks

In [ ]:
import os

In [ ]:
%%time
!python ./ez_enaminetor/csv_splitter.py Enamine_Diverse_REAL_drug-like_48.2M_cxsmiles.cxsmiles -c 1000000 -n enamine_real_dl_chunk

CPU times: user 241 ms, sys: 30.7 ms, total: 271 ms
Wall time: 28.8 s


In [ ]:
chunks = [i for i in os.listdir() if 'enamine_real_dl_chunk' in i]
print(f'number of chunks: {len(chunks)}')

number of chunks: 49


In [ ]:
%%time
!for i in enamine_real_dl_chunk*.csv; do python ./ez_enaminetor/csv2parquet_withProps.py --input $i --output ${i%.*}.pq | echo ${i%.*}; done

enamine_real_dl_chunk_10
enamine_real_dl_chunk_11
enamine_real_dl_chunk_12
enamine_real_dl_chunk_13
enamine_real_dl_chunk_14
enamine_real_dl_chunk_15
enamine_real_dl_chunk_16
enamine_real_dl_chunk_17
enamine_real_dl_chunk_18
enamine_real_dl_chunk_19
enamine_real_dl_chunk_1
enamine_real_dl_chunk_20
enamine_real_dl_chunk_21
enamine_real_dl_chunk_22
enamine_real_dl_chunk_23
enamine_real_dl_chunk_24
enamine_real_dl_chunk_25
enamine_real_dl_chunk_26
enamine_real_dl_chunk_27
enamine_real_dl_chunk_28
enamine_real_dl_chunk_29
enamine_real_dl_chunk_2
enamine_real_dl_chunk_30
enamine_real_dl_chunk_31
enamine_real_dl_chunk_32
enamine_real_dl_chunk_33
enamine_real_dl_chunk_34
enamine_real_dl_chunk_35
enamine_real_dl_chunk_36
enamine_real_dl_chunk_37
enamine_real_dl_chunk_38
enamine_real_dl_chunk_39
enamine_real_dl_chunk_3
enamine_real_dl_chunk_40
enamine_real_dl_chunk_41
enamine_real_dl_chunk_42
enamine_real_dl_chunk_43
enamine_real_dl_chunk_44
enamine_real_dl_chunk_45
enamine_real_dl_chunk_46
ena

In [67]:
pq.read_table('enamine_real_dl_chunk_45.pq')

pyarrow.Table
smiles: string
idnumber: string
Type: string
mw: double
hba: int64
hbd: int64
num_heavy_atoms: int64
----
smiles: [["CC(OC(F)F)C(=O)NC1=NC2=CC=CC(O)=C2S1","CC(OC(F)F)C(=O)NC1=NC=C(C(C)(C)O)S1","CC(OC(F)F)C(=O)NC1=NN(C)C(C#N)=C1","CC(OC(F)F)C(=O)NC1CC(CNC(=O)OC(C)(C)C)C1","CC(OC(F)F)C(=O)NC1CCN(C2CCN(C)C2=O)CC1",...,"CC1=C(C(=O)NC(C)C(C)NC(=O)C2=C(C)C(Br)=NO2)N=CS1","CC1=C(C(=O)NC(C)C(C)NC(=O)C2=C(C)N=C(C(C)(C)C)N=C2C)ON=C1","CC1=C(C(=O)NC(C)C(C)NC(=O)C2=C(C)NC(C)=C2C)N(C(C)C)N=C1","CC1=C(C(=O)NC(C)C(C)NC(=O)C2=C(Cl)CC2)NC(=O)N1","CC1=C(C(=O)NC(C)C(C)NC(=O)C2=C(F)NN=C2)C=NN1C"],["CC1=C(C(=O)NC(C)C(C)NC(=O)C2=CC(C(F)F)=CC(F)=C2)SN=C1","CC1=C(C(=O)NC(C)C(C)NC(=O)C2=COC=C2Br)C=NN1C","CC1=C(C(=O)NC(C)C(C)NC(=O)CC(C)(C)O)C(C)=C(Br)O1","CC1=C(C(=O)NC(C)C(N)C2=CC=CC=C2)C=CC=C1NC(C)C","CC1=C(C(=O)NC(C)C(N)C2=CC=CC=C2)SCCO1",...,"CC1=CC=C(C)C(CC(=O)NC2=NC=CC(O)=C2C)=C1","CC1=CC=C(C)C(CC(=O)NCCC2=NC=C(C)O2)=C1","CC1=CC=C(C)C(CC(=O)NCCC2CCC(C)C2)=C1","CC1=CC=C(C)C(CC(=O)OC2CC3(C2)CC(

In [101]:
# converting to IPC

def convert_parquet_to_ipc(parquet_path, ipc_path):
    # Read the parquet file
    table = pq.read_table(parquet_path)

    # Convert the table to IPC format
    writer = pa.RecordBatchFileWriter(ipc_path, table.schema)

    # Write the table to the IPC file
    for batch in table.to_batches():
        writer.write(batch)

    # Close the IPC writer
    writer.close()

In [104]:
chunks[0].split('.')[0]+ '.ipc'

'enamine_real_dl_chunk_48.ipc'

In [111]:
%%time
chunks = [i for i in os.listdir() if 'enamine_real_dl_chunk' in i]
chunks = [i for i in chunks if '.pq' in i]
for i in chunks:
  bname = i.split('.')[0]
  convert_parquet_to_ipc(i, bname + '.ipc')
  print(bname + '.ipc')

enamine_real_dl_chunk_48.ipc
enamine_real_dl_chunk_34.ipc
enamine_real_dl_chunk_42.ipc
enamine_real_dl_chunk_11.ipc
enamine_real_dl_chunk_22.ipc
enamine_real_dl_chunk_43.ipc
enamine_real_dl_chunk_41.ipc
enamine_real_dl_chunk_5.ipc
enamine_real_dl_chunk_3.ipc
enamine_real_dl_chunk_38.ipc
enamine_real_dl_chunk_10.ipc
enamine_real_dl_chunk_20.ipc
enamine_real_dl_chunk_47.ipc
enamine_real_dl_chunk_45.ipc
enamine_real_dl_chunk_8.ipc
enamine_real_dl_chunk_37.ipc
enamine_real_dl_chunk_31.ipc
enamine_real_dl_chunk_44.ipc
enamine_real_dl_chunk_25.ipc
enamine_real_dl_chunk_6.ipc
enamine_real_dl_chunk_14.ipc
enamine_real_dl_chunk_24.ipc
enamine_real_dl_chunk_13.ipc
enamine_real_dl_chunk_27.ipc
enamine_real_dl_chunk_7.ipc
enamine_real_dl_chunk_28.ipc
enamine_real_dl_chunk_29.ipc
enamine_real_dl_chunk_16.ipc
enamine_real_dl_chunk_23.ipc
enamine_real_dl_chunk_19.ipc
enamine_real_dl_chunk_9.ipc
enamine_real_dl_chunk_26.ipc
enamine_real_dl_chunk_12.ipc
enamine_real_dl_chunk_18.ipc
enamine_real_dl_chun

## Benchmark: read and filter

In [90]:
import os
import polars as pl
import pandas as pd
import dask.dataframe as dd
import time
import fnmatch
from tqdm import tqdm

import pyarrow.parquet as pq
import pyarrow as pa
from pyarrow import dataset as ds

In [80]:
def rflt_data_pandas(directory, pattern='*.pq'):
    start_time = time.time()
    row_count = 0

    files = fnmatch.filter(os.listdir(directory), pattern)
    for filename in tqdm(files, desc="Processing files"):
        df = pd.read_parquet(os.path.join(directory, filename))
        row_count += df[(df.hba > 6) & (df.hbd >= 2) & (df.num_heavy_atoms > 15) & (df.num_heavy_atoms < 30) & (df.mw > 222)].shape[0]

    elapsed_time = time.time() - start_time
    return row_count, elapsed_time

In [81]:
def rflt_data_polars(directory, pattern='*.pq'):
    start_time = time.time()
    row_count = 0
    files = fnmatch.filter(os.listdir(directory), pattern)
    for filename in tqdm(files, desc="Processing files"):
        df = pl.read_parquet(os.path.join(directory, filename))
        row_count += df.filter((df["hba"] > 6) & (df["hbd"] >= 2) & (df["num_heavy_atoms"] > 15) & (df["num_heavy_atoms"] < 30) & (df["mw"] > 222)).shape[0]

    elapsed_time = time.time() - start_time
    return row_count, elapsed_time

In [82]:
def rflt_data_dask(directory, pattern='*.pq'):
    start_time = time.time()

    df = dd.read_parquet(directory + '/' + pattern)
    df_filtered = df[(df.hba > 6) & (df.hbd >= 2) & (df.num_heavy_atoms > 15) & (df.num_heavy_atoms < 30) & (df.mw > 222)]

    row_count = df_filtered.shape[0].compute()
    elapsed_time = time.time() - start_time
    return row_count, elapsed_time

In [83]:
def rflt_data_pyarrow(directory, pattern='*.pq'):
    start_time = time.time()
    row_count = 0

    files = fnmatch.filter(os.listdir(directory), pattern)
    for filename in files:
        table = pq.read_table(os.path.join(directory, filename))

        mask_hba = [x > 6 for x in table.column('hba').to_pylist()]
        mask_hbd = [x >= 2 for x in table.column('hbd').to_pylist()]
        num_heavy_atoms_list = table.column('num_heavy_atoms').to_pylist()
        mask_num_heavy_atoms_1 = [x > 15 for x in num_heavy_atoms_list]
        mask_num_heavy_atoms_2 = [x < 30 for x in num_heavy_atoms_list]
        mask_mw = [x > 222 for x in table.column('mw').to_pylist()]

        final_mask = [hba and hbd and ha1 and ha2 and mw for hba, hbd, ha1, ha2, mw in zip(mask_hba, mask_hbd, mask_num_heavy_atoms_1, mask_num_heavy_atoms_2, mask_mw)]
        row_count += sum(final_mask)

    elapsed_time = time.time() - start_time
    return row_count, elapsed_time


In [97]:
def rflt_data_pyarrow3(directory_path, pattern):

    filters = [
    ('hba', '>', 6),
    ('hbd', '>=', 2),
    ('num_heavy_atoms', '>', 15),
    ('num_heavy_atoms', '<', 30),
    ('mw', '>', 222)]

    files = fnmatch.filter(os.listdir(directory), pattern)

    total_rows = 0
    start_time = time.time()

    # Apply filters to each file
    for filename in files:
        dataset = ds.dataset(filename, format='parquet')

        filter_expression = None
        for column, operator, value in filters:
            filter_condition = ds.field(column) > value
            if filter_expression is None:
                filter_expression = filter_condition
            else:
                filter_expression = filter_expression & filter_condition

        filtered_table = dataset.to_table(filter=filter_expression)
        total_rows += len(filtered_table)

    end_time = time.time()
    time_taken = end_time - start_time

    return total_rows, time_taken

In [112]:
def rflt_data_ipc(directory_path, pattern):

    filters = [
    ('hba', '>', 6),
    ('hbd', '>=', 2),
    ('num_heavy_atoms', '>', 15),
    ('num_heavy_atoms', '<', 30),
    ('mw', '>', 222)]

    files = fnmatch.filter(os.listdir(directory), pattern)

    total_rows = 0
    start_time = time.time()

    # Apply filters to each file
    for filename in files:
        dataset = ds.dataset(filename, format='ipc')

        filter_expression = None
        for column, operator, value in filters:
            filter_condition = ds.field(column) > value
            if filter_expression is None:
                filter_expression = filter_condition
            else:
                filter_expression = filter_expression & filter_condition

        filtered_table = dataset.to_table(filter=filter_expression)
        total_rows += len(filtered_table)

    end_time = time.time()
    time_taken = end_time - start_time

    return total_rows, time_taken

In [32]:
pd_rows, pd_time = rflt_data_pandas('.', 'enamine_real_dl_chunk*.pq')

Processing files: 100%|██████████| 49/49 [00:47<00:00,  1.03it/s]


In [35]:
pl_rows, pl_time = rflt_data_polars('.', 'enamine_real_dl_chunk*.pq')

Processing files: 100%|██████████| 49/49 [00:06<00:00,  7.47it/s]


In [43]:
dd_rows, dd_time = rflt_data_dask('.', 'enamine_real_dl_chunk*.pq')

In [109]:
pa_rows, pa_time = rflt_data_pyarrow3('.', 'enamine_real_dl_chunk*.pq')

In [114]:
pai_rows, pai_time = rflt_data_ipc('.', 'enamine_real_dl_chunk*.ipc')

In [115]:
print(f'Pandas: {pd_rows} rows, {pd_time:.2f} seconds')
print(f'Polars: {pl_rows} rows, {pl_time:.2f} seconds')
print(f'Dask: {dd_rows} rows, {dd_time:.2f} seconds')
print(f'PyArrow Parquet: {pa_rows} rows, {pa_time:.2f} seconds')
print(f'PyArrow IPC: {pai_rows} rows, {pai_time:.2f} seconds')

Pandas: 442912 rows, 47.49 seconds
Polars: 442912 rows, 6.57 seconds
Dask: 442912 rows, 56.60 seconds
PyArrow Parquet: 613 rows, 4.31 seconds
PyArrow IPC: 613 rows, 1.53 seconds
